In [1]:
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/


In [2]:
from __future__ import annotations

EXP = "exp001"
TARGET_MODEL = "/kaggle/input/llama2-7b-hf/Llama2-7b-hf"

DEBUG = False

In [3]:
# %% Directory settings

# ====================================================
# Directory settings
# ====================================================
from pathlib import Path

OUTPUT_DIR = Path("/kaggle/input/llm-detect-ai-generated-text-llama-2-13b")
#OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("../input/")

In [4]:
import pandas as pd
test_df = pd.read_csv(INPUT_DIR / "llm-detect-ai-generated-text/test_essays.csv", sep=',')
print(f'test_df.shape: {test_df.shape}')


test_df.shape: (3, 3)


In [5]:
test_df.head(3)

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [6]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# datasets
from datasets import Dataset

# from pandas
test_ds = Dataset.from_pandas(test_df)


In [8]:
def preprocess_function(examples, max_length=1024):
    return tokenizer(examples["text"], truncation=True, max_length=max_length)

In [9]:
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
# load model / tokenizer with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=2,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/llama2-7b-hf/Llama2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model = PeftModel.from_pretrained(base_model, str(OUTPUT_DIR))


In [14]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [15]:
pred_output = trainer.predict(test_tokenized_ds)
logits = pred_output.predictions

In [16]:
from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
probs = sigmoid(logits[:, 1])
probs

array([0.02179, 0.02263, 0.02434], dtype=float16)

In [17]:
sub = pd.DataFrame()
sub['id'] = test_df['id']
sub['generated'] = probs
sub.to_csv('submission.csv', index=False)
sub.head()

,id,generated
0,0000aaaa,0.021790
1,1111bbbb,0.022629
2,2222cccc,0.024338
